This notebook is to fine tune a mask2former model, and assumes transformer datasets are already available 

In [ ]:
%pip install -U -q transformers[torch] evaluate timm albumentations accelerate hugggingface_hub

May need to restart kernel after installs

In [4]:
import json
import torch
import datasets
import requests
import evaluate
import numpy as np
import huggingface_hub
from PIL import Image
import albumentations as A
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from typing import Tuple, Any
from dataclasses import dataclass
from datasets import load_dataset, load_from_disk
import os
import matplotlib.patches as mpatches
from huggingface_hub import hf_hub_download
from torch.utils.data import Dataset, DataLoader
from transformers import (
    MaskFormerImageProcessor,
    AutoImageProcessor,
    MaskFormerForInstanceSegmentation,
)
from transformers import MaskFormerConfig

torch.manual_seed(42)

In [ ]:
# Get the correct ontology for the training dataset - first slot needs to be 'unlabeled'
# T2 ontology names from Labelbox - with 'unlabeled' in first slot
names = [
 'unlabeled',
 'C1 Def/Lift',
 'C2 Missing',
 'AI Air Inf',
 'TB Th Brdge',
 'CW1 Def/Missing',
 'CW3 Th Byp',
 'CW4 Top Mult',
 'CW5 Bot Mult',
 'B Boxing',
 'TW1 Def/Lift',
 'TW2 Missing',
 'F1 Def/Missing',
 'R Refl',
 'O Openg',
 'LH Loft',
 'TP Tripod']

id2label = {i: name for i, name in enumerate(names)}
label2id = {v: k for k, v in id2label.items()}
print(id2label)

In [ ]:
# Connect to the training dataset which is assumed to be on g-drive
from google.colab import drive
drive.mount('/content/drive')

im_folder = r'/content/drive/My Drive/Data/Labelbox_Data/Fiesta250_seg_T2_test_to_train/expanded_dataset3'

#hf_train_ds = load_from_disk(os.path.join(im_folder,'hf_train_FiestaTTTSeg'))
#hf_val_ds = load_from_disk(os.path.join(im_folder,'hf_eval_FiestaTTTSeg'))
dataset_maskformer = load_from_disk(os.path.join(im_folder, 'dataset_maskformer'))

train_ds = dataset_maskformer['train']
val_ds = dataset_maskformer['val']